# Call SageMaker Endpoint - Simple Version

Minimal setup to call your deployed GPT-2 model for financial law analysis.

## 1. Set Credentials

Get fresh credentials from `api_hf.ipynb` (last cell) and paste here:

In [ ]:
import os

# PASTE CREDENTIALS FROM SAGEMAKER HERE:
os.environ['AWS_ACCESS_KEY_ID'] = 'YOUR_ACCESS_KEY_ID_HERE'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'YOUR_SECRET_ACCESS_KEY_HERE'
os.environ['AWS_SESSION_TOKEN'] = 'YOUR_SESSION_TOKEN_HERE'
os.environ['AWS_DEFAULT_REGION'] = 'us-west-2'

print("✓ Credentials set")

✓ Credentials set


## 2. Call the Endpoint

In [18]:
import boto3
import json

# Setup
runtime = boto3.client('sagemaker-runtime', region_name='us-west-2')
endpoint_name = "huggingface-demo-endpoint"

# Your law document
law_text = """
New regulations require all financial institutions to maintain 
higher capital reserves of at least 15% by Q2 2026.
"""

# Call endpoint
response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='application/json',
    Body=json.dumps({
        "inputs": f"Analyze this law for financial impact: {law_text}",
        "parameters": {"max_new_tokens": 200}
    })
)

# Print result
result = json.loads(response['Body'].read().decode())
print(result)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:15                                                                                   │
│                                                                                                  │
│   12 """                                                                                         │
│   13                                                                                             │
│   14 # Call endpoint                                                                             │
│ ❱ 15 response = runtime.invoke_endpoint(                                                         │
│   16 │   EndpointName=endpoint_name,                                                             │
│   17 │   ContentType='application/json',                                                         │
│   18 │   Body=json.dumps({                                                                       │
│                                                                                                  │
│ /home/letua/PythonProjects/Datathon/Datathon-2025/venv/lib/python3.12/site-packages/botocore/cli │
│ ent.py:602 in _api_call                                                                          │
│                                                                                                  │
│    599 │   │   │   │   │   f"{py_operation_name}() only accepts keyword arguments."              │
│    600 │   │   │   │   )                                                                         │
│    601 │   │   │   # The "self" in this scope is referring to the BaseClient.                    │
│ ❱  602 │   │   │   return self._make_api_call(operation_name, kwargs)                            │
│    603 │   │                                                                                     │
│    604 │   │   _api_call.__name__ = str(py_operation_name)                                       │
│    605                                                                                           │
│                                                                                                  │
│ /home/letua/PythonProjects/Datathon/Datathon-2025/venv/lib/python3.12/site-packages/botocore/con │
│ text.py:123 in wrapper                                                                           │
│                                                                                                  │
│   120 │   │   │   with start_as_current_context():                                               │
│   121 │   │   │   │   if hook:                                                                   │
│   122 │   │   │   │   │   hook()                                                                 │
│ ❱ 123 │   │   │   │   return func(*args, **kwargs)                                               │
│   124 │   │                                                                                      │
│   125 │   │   return wrapper                                                                     │
│   126                                                                                            │
│                                                                                                  │
│ /home/letua/PythonProjects/Datathon/Datathon-2025/venv/lib/python3.12/site-packages/botocore/cli │
│ ent.py:1078 in _make_api_call                                                                    │
│                                                                                                  │
│   1075 │   │   │   │   'error_code_override'                                                     │
│   1076 │   │   │   ) or error_info.get("Code")                                                   │
│   1077 │   │   │   error_class = self.exceptions.from_code(error_code)                           │
│ ❱ 1078 │   │   │   raise error_class(parsed_response, opera

## 3. Reusable Function (Optional)

In [4]:
def analyze_law(law_text):
    """Analyze a law document"""
    response = runtime.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType='application/json',
        Body=json.dumps({
            "inputs": f"Analyze impact on industries and stocks: {law_text}",
            "parameters": {"max_new_tokens": 200}
        })
    )
    return json.loads(response['Body'].read().decode())

# Test
result = analyze_law("Banks must increase capital reserves by 15%")
print(result)

[{'generated_text': "Analyze impact on industries and stocks: Banks must increase capital reserves by 15% to compensate for the negative impact of high credit ratings.\n\nIn order to provide a more balanced picture, the Bank of England and the City of London (BIS) are set to launch a comprehensive research programme beginning in mid-2019 to help banks and other businesses to reduce excessive borrowing costs. Data on the impact on banks will be gathered by the Financial Statistics Authority (SFSA) and will be put to use to create a framework to monitor the impact on both businesses and the economy, the BIS explains.\n\nThere is an emphasis placed on the following:\n\nEnabling Bank of England to reduce excessive borrowing costs to prevent unnecessary redundancies.\n\nConducting an independent analysis of banks' capital expenditure, including their financial conditions, to gauge the impact on banks' operating capacity, to assess the feasibility of raising capital by the rate of change or 

---

## 4. Working with S3 Buckets

Access files stored in S3 (useful for law documents, results, etc.)

In [ ]:
# Setup S3 client
s3 = boto3.client('s3')

# Your hackathon bucket structure:
# Bucket name: YOUR-BUCKET-NAME
# Your project prefix: YOUR-PROJECT-PREFIX/

default_bucket = "YOUR-BUCKET-NAME"
project_prefix = "YOUR-PROJECT-PREFIX/"

print(f"Bucket: {default_bucket}")
print(f"Project path: s3://{default_bucket}/{project_prefix}")

# Verify access
try:
    s3.head_bucket(Bucket=default_bucket)
    print("✓ Bucket is accessible")
    
    # List your project files
    response = s3.list_objects_v2(Bucket=default_bucket, Prefix=project_prefix, MaxKeys=10)
    if 'Contents' in response:
        print(f"\n✓ Found {len(response['Contents'])} items in your project folder")
    else:
        print("\n⚠ Project folder is empty or not accessible")
except Exception as e:
    print(f"⚠ Error: {e}")

Bucket: csv-file-store-eae75cc0
Project path: s3://csv-file-store-eae75cc0/dzd-5hiypewhfajt1s/bsy8e2eigl2nio/
✓ Bucket is accessible

✓ Found 10 items in your project folder


In [12]:
# Upload a file to S3
def upload_to_s3(file_path, bucket_name, s3_key):
    """Upload file to S3 bucket"""
    s3.upload_file(file_path, bucket_name, s3_key)
    print(f"✓ Uploaded {file_path} to s3://{bucket_name}/{s3_key}")

# Example:
# upload_to_s3("law_document.pdf", "my-bucket", "documents/law_document.pdf")

In [13]:
# Download a file from S3
def download_from_s3(bucket_name, s3_key, local_path):
    """Download file from S3 bucket"""
    s3.download_file(bucket_name, s3_key, local_path)
    print(f"✓ Downloaded s3://{bucket_name}/{s3_key} to {local_path}")

# Example:
# download_from_s3("my-bucket", "documents/law_document.pdf", "downloaded_law.pdf")

In [14]:
# Read file directly from S3 (without downloading)
def read_s3_file(bucket_name, s3_key):
    """Read text file content from S3"""
    obj = s3.get_object(Bucket=bucket_name, Key=s3_key)
    content = obj['Body'].read().decode('utf-8')
    return content

# Example - Read law document from S3 and analyze it:
# law_text = read_s3_file("my-bucket", "documents/law_document.txt")
# result = analyze_law(law_text)
# print(result)

In [15]:
# List files in a bucket
def list_s3_files(bucket_name, prefix=""):
    """List all files in an S3 bucket with optional prefix"""
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
    
    if 'Contents' in response:
        print(f"Files in s3://{bucket_name}/{prefix}:")
        for obj in response['Contents']:
            print(f"  - {obj['Key']} ({obj['Size']} bytes)")
    else:
        print(f"No files found in s3://{bucket_name}/{prefix}")

# Example:
# list_s3_files("my-bucket", "documents/")

In [16]:
# List all files in your project folder
list_s3_files(default_bucket, project_prefix)

# Or list files in a specific subfolder:
# list_s3_files(default_bucket, f"{project_prefix}shared/DATA/")

Files in s3://csv-file-store-eae75cc0/dzd-5hiypewhfajt1s/bsy8e2eigl2nio/:
  - dzd-5hiypewhfajt1s/bsy8e2eigl2nio/dev/ (0 bytes)
  - dzd-5hiypewhfajt1s/bsy8e2eigl2nio/shared/ (0 bytes)
  - dzd-5hiypewhfajt1s/bsy8e2eigl2nio/shared/.git/ (0 bytes)
  - dzd-5hiypewhfajt1s/bsy8e2eigl2nio/shared/.git/COMMIT_EDITMSG (16 bytes)
  - dzd-5hiypewhfajt1s/bsy8e2eigl2nio/shared/.git/FETCH_HEAD (0 bytes)
  - dzd-5hiypewhfajt1s/bsy8e2eigl2nio/shared/.git/HEAD (23 bytes)
  - dzd-5hiypewhfajt1s/bsy8e2eigl2nio/shared/.git/branches/ (0 bytes)
  - dzd-5hiypewhfajt1s/bsy8e2eigl2nio/shared/.git/config (270 bytes)
  - dzd-5hiypewhfajt1s/bsy8e2eigl2nio/shared/.git/description (73 bytes)
  - dzd-5hiypewhfajt1s/bsy8e2eigl2nio/shared/.git/hooks/ (0 bytes)
  - dzd-5hiypewhfajt1s/bsy8e2eigl2nio/shared/.git/hooks/applypatch-msg.sample (478 bytes)
  - dzd-5hiypewhfajt1s/bsy8e2eigl2nio/shared/.git/hooks/commit-msg.sample (896 bytes)
  - dzd-5hiypewhfajt1s/bsy8e2eigl2nio/shared/.git/hooks/fsmonitor-watchman.sample (4655 

### Complete Workflow Example: S3 → Analysis → S3

In [10]:
# Complete pipeline: Read from S3 → Analyze → Save results to S3
def analyze_law_from_s3(input_bucket, input_key, output_bucket, output_key):
    """
    Read law document from S3, analyze it, save results back to S3
    """
    # 1. Read document from S3
    print(f"Reading from s3://{input_bucket}/{input_key}")
    law_text = read_s3_file(input_bucket, input_key)
    
    # 2. Analyze with SageMaker endpoint
    print("Analyzing with GPT-2...")
    result = analyze_law(law_text)
    
    # 3. Save results to S3
    print(f"Saving results to s3://{output_bucket}/{output_key}")
    s3.put_object(
        Bucket=output_bucket,
        Key=output_key,
        Body=json.dumps(result, indent=2),
        ContentType='application/json'
    )
    
    print("✓ Complete!")
    return result

# Example:
# result = analyze_law_from_s3(
#     input_bucket="my-bucket",
#     input_key="laws/banking_regulation.txt",
#     output_bucket="my-bucket",
#     output_key="results/banking_analysis.json"
# )